In [1]:
import pandas as pd
import time
import numpy as np
import datetime
from datetime import date
from datetime import datetime

In [9]:
#Devices
df_D = pd.read_excel(r'C:\Users\edsantil2101\Downloads\TVsets_status 0820.xlsx') #IBIS: AZRA TV SETS STATUS
#Sources
df_S = pd.read_excel(r'C:\Users\edsantil2101\Downloads\source_list 0820.xlsx') #IBIS: MAX SOURCE LIST
#SM Installed
df_SM = pd.read_excel(r'C:\Users\edsantil2101\Downloads\sm 0820.xlsx') 

date_today = datetime.now().strftime("%Y%m%d")

RowsDrop = list(range(0,2))
RowsDrop1 = list(range(0,8))

_=df_D.drop(RowsDrop).reset_index(drop=True)
df=_.rename(columns=_.iloc[0]).drop(_.index[0])

d = df_S.drop(RowsDrop).reset_index(drop=True)
dfs1=d.rename(columns=d.iloc[0]).drop(d.index[0])

a=df_SM.drop(RowsDrop1).reset_index(drop=True)
SM=a.rename(columns=a.iloc[0]).drop(a.index[0])
SM = SM.drop(SM.columns[0], axis=1)
SM.rename(columns = {'Family Code':'fam.familycode'}, inplace = True)

dfs1['Designation'] = dfs1['src.srctype'].str[-4:]

key = {'Designation': ['SMTV','FADT','CATV','DSSD','DSPS','DSON','DSAM','MONR','IPTV','IPVD','IPVA','IPGX','IPJW','IPBO','IPAM','IPON','IPBN','DCAB','DSBN','DSON','DSAD','DSMH','DSGX','DSBO','DSPO','DSAT','FADX','CAST','APPL','AMZF','ANRD','PLST','XBOX','GAME','DVDR','COMP'],
        'Platforms': ['IPTV/ Internet/ Streaming','Digital Terrestrial','Cable','Satellite','Satellite','Satellite','Satellite','Unknown','IPTV/ Internet/ Streaming','IPTV/ Internet/ Streaming','IPTV/ Internet/ Streaming','IPTV/ Internet/ Streaming','IPTV/ Internet/ Streaming','IPTV/ Internet/ Streaming','IPTV/ Internet/ Streaming','IPTV/ Internet/ Streaming','IPTV/ Internet/ Streaming','Cable','Satellite','Satellite','Satellite','Satellite','Satellite','Satellite','Satellite','Satellite','Digital Terrestrial','IPTV/ Internet/ Streaming','IPTV/ Internet/ Streaming','IPTV/ Internet/ Streaming','IPTV/ Internet/ Streaming','IPTV/ Internet/ Streaming','IPTV/ Internet/ Streaming','IPTV/ Internet/ Streaming','IPTV/ Internet/ Streaming','IPTV/ Internet/ Streaming']}

df_key = pd.DataFrame(key)

dfs = dfs1.merge(df_key, on = 'Designation', how = 'left')

report = []

df['unit.unitid'] = pd.to_numeric(df['unit.unitid'])

conditions = [(df['unit.unitid'] < 21),
              (df['unit.unitid'] > 20) &  (df['unit.unitid'] < 40),
              (df['unit.unitid'] > 39) & (df['unit.unitid'] < 60),
              (df['unit.unitid'] > 59)]

choices = ['TV','laptop','tablet','smartphone']

df['device type'] = np.select(conditions, choices, default=0)

df['fam.familycode'] = df['fam.familycode'].astype(str) #we need this to count the duplicate values properly
df['unit.unitid'] = df['unit.unitid'].astype(float)
#cond=((df['device type']=='TV') & (df['unit.unitid']<16) & (df['fam.status'].isin(['120-Running In (Not to Produce)','130-To be Polled and Produced','140-Withheld(Not to Produce)','150-Off Directory(Not Polling at all)','155-Forced In(Holliday-OFF DIRECTORY to Produce)'])))

#First pivot Table
filters1= (df['device type']=='TV') & (df['unit.unitid']<16) & (df['fam.status'].astype(str).str.startswith('12')|df['fam.status'].astype(str).str.startswith('13')|df['fam.status'].astype(str).str.startswith('14')|df['fam.status'].astype(str).str.startswith('15'))
TV=pd.pivot_table(df.loc[filters1],values=['fam.familycode'],index=['unit.status','unit.secstatus'],aggfunc=np.count_nonzero, margins = True, observed=True)
TV_Totals = TV.groupby(by=['unit.status']).sum() #Auxiliar table

filters2= (df['device type'] !='TV') & (df['unit.unitid']>15) & (df['fam.status'].astype(str).str.startswith('12')|df['fam.status'].astype(str).str.startswith('13')|df['fam.status'].astype(str).str.startswith('14')|df['fam.status'].astype(str).str.startswith('15'))
#pd.pivot_table(df,values=['unit.unitid'],index=['unit.status', 'unit.secstatus'], columns=['device type'],aggfunc=np.count_nonzero,margins=True, observed=True)
#Secon pivot table
PT=pd.pivot_table(df.loc[filters2],values=['unit.unitid'],index=['unit.status','unit.secstatus'],columns=['device type'],aggfunc='count',margins=True, observed=True)
PT2 = pd.pivot_table(df.loc[filters2],values=['unit.unitid'],index=['unit.status'],aggfunc='count',margins=True, observed=True) #Auxiliar table

#Auxiliar tables by device

filtersLaptop= (df['device type'] == 'laptop') & (df['unit.unitid']>15) & (df['fam.status'].astype(str).str.startswith('12')|df['fam.status'].astype(str).str.startswith('13')|df['fam.status'].astype(str).str.startswith('14')|df['fam.status'].astype(str).str.startswith('15'))
filtersSP= (df['device type'] =='smartphone') & (df['unit.unitid']>15) & (df['fam.status'].astype(str).str.startswith('12')|df['fam.status'].astype(str).str.startswith('13')|df['fam.status'].astype(str).str.startswith('14')|df['fam.status'].astype(str).str.startswith('15'))
filtersTablet= (df['device type'] =='tablet') & (df['unit.unitid']>15) & (df['fam.status'].astype(str).str.startswith('12')|df['fam.status'].astype(str).str.startswith('13')|df['fam.status'].astype(str).str.startswith('14')|df['fam.status'].astype(str).str.startswith('15'))
LP=pd.pivot_table(df.loc[filtersLaptop],values=['unit.unitid'],index=['unit.secstatus'],aggfunc='count',margins=True, observed=True)
SP=pd.pivot_table(df.loc[filtersSP],values=['unit.unitid'],index=['unit.secstatus'],aggfunc='count',margins=True, observed=True)
TB=pd.pivot_table(df.loc[filtersTablet],values=['unit.unitid'],index=['unit.secstatus'],aggfunc='count',margins=True, observed=True)
LP2=pd.pivot_table(df.loc[filtersLaptop],values=['unit.unitid'],index=['unit.status'],aggfunc='count',margins=True, observed=True)
SP2=pd.pivot_table(df.loc[filtersSP],values=['unit.unitid'],index=['unit.status'],aggfunc='count',margins=True, observed=True)
TB2=pd.pivot_table(df.loc[filtersTablet],values=['unit.unitid'],index=['unit.status'],aggfunc='count',margins=True, observed=True)

df = df.merge(SM[['fam.familycode','Value']].astype(str), on = 'fam.familycode', how = 'left')
df.rename(columns = {'Value':'SM install status'}, inplace = True)

#3ra tabla dinámica
filters3 = (df['fam.status'].astype(str).str.startswith('12')|df['fam.status'].astype(str).str.startswith('13')|df['fam.status'].astype(str).str.startswith('14')|df['fam.status'].astype(str).str.startswith('15'))
YN = pd.pivot_table(df.loc[filters3],values=['fam.familycode'],index=['SM install status'],aggfunc=pd.Series.nunique,margins=True, observed=True)

#4ta tabla dinámica
filtersS = (dfs['unit.status'] == '0-Monitored')
Sources = pd.pivot_table(dfs.loc[filtersS],values=['fam.familycode'],index=['Platforms','src.srctype'],aggfunc='count',margins=True, observed=True)
Sources_Totals = Sources.groupby(by=['Platforms']).sum()

#Extrayendo todos los datos para el reporte final

R1 = TV_Totals.loc['All','fam.familycode']
R2 = TV_Totals.loc['0-Monitored','fam.familycode']
R3 = TV_Totals.loc['1-Temporarily unmonitored','fam.familycode']
R4 = TV_Totals.loc['2-In existance but not be monitored','fam.familycode']
R5 = TV_Totals.loc['3-No longer used','fam.familycode']
R6 = LP.loc['502-PC/Laptop - Sucessfully Mapped','unit.unitid'] + TB.loc['602-Tablet - Successfully Mapped','unit.unitid'] + SP.loc['702-Smartphone - Successfully mapped','unit.unitid']
R7 = LP.loc['503-PC/Laptop - Pending MAC/ODID Mapping','unit.unitid'] + TB.loc['603-Tablet - Pending MAC/ODID Mapping','unit.unitid'] + SP.loc['703-Smartphone - Pending MAC/ODID Mapping','unit.unitid']
R8 = LP.loc['501-PC/Laptop - Waiting Mapping/Registration','unit.unitid'] + TB.loc['601-Tablet-Waiting Mapping/Registration','unit.unitid'] + SP.loc['701-Smartphone-Waiting Mapping/Registration','unit.unitid']
R9 = PT2.loc['8-Digital Device Not Installed','unit.unitid']
R10 = LP.loc['502-PC/Laptop - Sucessfully Mapped','unit.unitid']
R11 = LP.loc['503-PC/Laptop - Pending MAC/ODID Mapping','unit.unitid']
R12 = LP.loc['501-PC/Laptop - Waiting Mapping/Registration','unit.unitid']
R13 = LP2.loc['8-Digital Device Not Installed','unit.unitid']
R14 = TB.loc['602-Tablet - Successfully Mapped','unit.unitid']
R15 = TB.loc['603-Tablet - Pending MAC/ODID Mapping','unit.unitid']
R16 = TB.loc['601-Tablet-Waiting Mapping/Registration','unit.unitid']
R17 = TB2.loc['8-Digital Device Not Installed','unit.unitid']
R18 = SP.loc['702-Smartphone - Successfully mapped','unit.unitid']
R19 = SP.loc['703-Smartphone - Pending MAC/ODID Mapping','unit.unitid']
R20 = SP.loc['701-Smartphone-Waiting Mapping/Registration','unit.unitid']
R21 = SP2.loc['8-Digital Device Not Installed','unit.unitid']
R22 = YN.loc['1 - Yes','fam.familycode']
R23 = Sources_Totals.loc['Digital Terrestrial','fam.familycode']
R24 = Sources_Totals.loc['IPTV/ Internet/ Streaming','fam.familycode']
R25 = Sources_Totals.loc['Satellite','fam.familycode']
R26 = Sources_Totals.loc['Cable','fam.familycode']

for r in range(1,27):
    report.append(locals()["R"+str(r)])
    
#Creando el dataframe del reporte final
data=[{'Nr. Total TVs':R1,'Nr. Monitored TVs/Metered':R2,'Nr. Temp Unmonitored TVs (Broken)':R3,'Nr. No Monitored - Portable/Only for non-broadcast (DVD etc.)':R4,'Nr. No Monitored - Not used anymore':R5,'Nr Digital Devices Measured':R6,'Nr Digital Devices Temp  Not Measured':R7,'Nr Digital Devices To be Mapped':R8,'Nr Digital Devices Not installed (other reason)':R9,'PC/Laptop - Successfully Mapped':R10,'PC/Laptop - Temp  Not Measured':R11,'PC/Laptop - To be Mapped':R12,'PC/Laptop - Not installed (other reason)':R13,'Tablet - Successfully Mapped':R14,'Tablet - Temp  Not Measured':R15,'Tablet - To be Mapped':R16,'Tablet - Not installed (other reason)':R17,'Smartphone - Successfully Mapped':R18,'Smartphone - Temp  Not Measured':R19,'Smartphone - To be Mapped':R20,'Smartphone - Not installed (other reason)':R21,'Streaming Meter Installed Y/N':R22,'Digital Terrestrial (Y/N)':R23,'IPTV / Internet/Streaming (Y/N)':R24,'Satellite (Y/N)':R25,'Cable (Y/N)':R26}]
df_RF=pd.DataFrame(data, index = ['Total'])

writer = pd.ExcelWriter(r'C:\Users\edsantil2101\Python\{date_today}_Devices_W33.xlsx')  #path + output_file_name, index = False
df.to_excel(writer,'Historical - Tv units')
dfs.to_excel(writer,'TV_Sources')
df_key.to_excel(writer,'Key')
TV.to_excel(writer,'Pivot 1')
PT.to_excel(writer, 'Pivot 2')
# df_temp_out6.to_excel(writer, 'Output6')
YN.to_excel(writer, 'Pivot 3')
Sources.to_excel(writer, 'Pivot 4')
df_RF.to_excel(writer, 'Devices Registered & Measured')
#df_check10.to_excel(writer, 'Output10')
writer.save()

C:\Users\edsantil2101\Anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [10]:
Sources_Totals

,fam.familycode
Platforms,
All,5314
IPTV/ Internet/ Streaming,2827
Satellite,2487


In [2]:
'''
path_out=r'C:\Users\edsantil2101\Python/'

#Devices
df_D = pd.read_csv(r'C:\Users\edsantil2101\Downloads\Copy of Devices Registered & Measured-List  Historical 20220625 W25 - Tv units Historical 20220625.csv')
#Sources
df_S = pd.read_csv(r'C:\Users\edsantil2101\Downloads\Copy of Devices Registered & Measured-List  Historical 20220625 W25 - Copy of TV_Sources_Histrorical of 20220525 (1).csv')

date_today = datetime.now().strftime("%Y%m%d")

#Formating both dataframes

RowsDrop = list(range(0,2))
_=df_D.drop(RowsDrop).reset_index(drop=True)
df=_.rename(columns=_.iloc[0]).drop(_.index[0])

d = df_S.drop(RowsDrop).reset_index(drop=True)
dfs1=d.rename(columns=d.iloc[0]).drop(d.index[0])

key = {'Designation': ['SMTV','FADT','CATV','DSSD','DSPS','DSON','DSAM','MONR','IPTV','IPVD','IPVA','IPGX','IPJW','IPBO','IPAM','IPON','IPBN','DCAB','DSBN','DSON','DSAD','DSMH','DSGX','DSBO','DSPO','DSAT','FADX','CAST','APPL','AMZF','ANRD','PLST','XBOX','GAME','DVDR','COMP'],
        'Platforms': ['IPTV/ Internet/ Streaming','Digital Terrestrial','Cable','Satellite','Satellite','Satellite','Satellite','Unknown','IPTV/ Internet/ Streaming','IPTV/ Internet/ Streaming','IPTV/ Internet/ Streaming','IPTV/ Internet/ Streaming','IPTV/ Internet/ Streaming','IPTV/ Internet/ Streaming','IPTV/ Internet/ Streaming','IPTV/ Internet/ Streaming','IPTV/ Internet/ Streaming','Cable','Satellite','Satellite','Satellite','Satellite','Satellite','Satellite','Satellite','Satellite','Digital Terrestrial','IPTV/ Internet/ Streaming','IPTV/ Internet/ Streaming','IPTV/ Internet/ Streaming','IPTV/ Internet/ Streaming','IPTV/ Internet/ Streaming','IPTV/ Internet/ Streaming','IPTV/ Internet/ Streaming','IPTV/ Internet/ Streaming','IPTV/ Internet/ Streaming']}
df_key = pd.DataFrame(key)

dfs1['Designation'] = dfs1['src.srctype'].str[-4:]
dfs = dfs1.merge(df_key, on = 'Designation', how = 'left')

report = []

df['fam.familycode'] = df['fam.familycode'].astype(str) #we need this to count the duplicate values properly
df['unit.unitid'] = df['unit.unitid'].astype(float)
#cond=((df['device type']=='TV') & (df['unit.unitid']<16) & (df['fam.status'].isin(['120-Running In (Not to Produce)','130-To be Polled and Produced','140-Withheld(Not to Produce)','150-Off Directory(Not Polling at all)','155-Forced In(Holliday-OFF DIRECTORY to Produce)'])))

#First pivot Table
filters1= (df['device type']=='TV') & (df['unit.unitid']<16) & (df['fam.status'].astype(str).str.startswith('12')|df['fam.status'].astype(str).str.startswith('13')|df['fam.status'].astype(str).str.startswith('14')|df['fam.status'].astype(str).str.startswith('15'))
TV=pd.pivot_table(df.loc[filters1],values=['fam.familycode'],index=['unit.status','unit.secstatus'],aggfunc=np.count_nonzero, margins = True, observed=True)
TV_Totals = TV.groupby(by=['unit.status']).sum() #Auxiliar table

filters2= (df['device type'] !='TV') & (df['unit.unitid']>15) & (df['fam.status'].astype(str).str.startswith('12')|df['fam.status'].astype(str).str.startswith('13')|df['fam.status'].astype(str).str.startswith('14')|df['fam.status'].astype(str).str.startswith('15'))
#pd.pivot_table(df,values=['unit.unitid'],index=['unit.status', 'unit.secstatus'], columns=['device type'],aggfunc=np.count_nonzero,margins=True, observed=True)
#Secon pivot table
PT=pd.pivot_table(df.loc[filters2],values=['unit.unitid'],index=['unit.status','unit.secstatus'],columns=['device type'],aggfunc='count',margins=True, observed=True)
PT2 = pd.pivot_table(df.loc[filters2],values=['unit.unitid'],index=['unit.status'],aggfunc='count',margins=True, observed=True) #Auxiliar table

#Auxiliar tables by device

filtersLaptop= (df['device type'] == 'laptop') & (df['unit.unitid']>15) & (df['fam.status'].astype(str).str.startswith('12')|df['fam.status'].astype(str).str.startswith('13')|df['fam.status'].astype(str).str.startswith('14')|df['fam.status'].astype(str).str.startswith('15'))
filtersSP= (df['device type'] =='smartphone') & (df['unit.unitid']>15) & (df['fam.status'].astype(str).str.startswith('12')|df['fam.status'].astype(str).str.startswith('13')|df['fam.status'].astype(str).str.startswith('14')|df['fam.status'].astype(str).str.startswith('15'))
filtersTablet= (df['device type'] =='tablet') & (df['unit.unitid']>15) & (df['fam.status'].astype(str).str.startswith('12')|df['fam.status'].astype(str).str.startswith('13')|df['fam.status'].astype(str).str.startswith('14')|df['fam.status'].astype(str).str.startswith('15'))
LP=pd.pivot_table(df.loc[filtersLaptop],values=['unit.unitid'],index=['unit.secstatus'],aggfunc='count',margins=True, observed=True)
SP=pd.pivot_table(df.loc[filtersSP],values=['unit.unitid'],index=['unit.secstatus'],aggfunc='count',margins=True, observed=True)
TB=pd.pivot_table(df.loc[filtersTablet],values=['unit.unitid'],index=['unit.secstatus'],aggfunc='count',margins=True, observed=True)
LP2=pd.pivot_table(df.loc[filtersLaptop],values=['unit.unitid'],index=['unit.status'],aggfunc='count',margins=True, observed=True)
SP2=pd.pivot_table(df.loc[filtersSP],values=['unit.unitid'],index=['unit.status'],aggfunc='count',margins=True, observed=True)
TB2=pd.pivot_table(df.loc[filtersTablet],values=['unit.unitid'],index=['unit.status'],aggfunc='count',margins=True, observed=True)

#3rd pivot table
filters3 = (df['fam.status'].astype(str).str.startswith('12')|df['fam.status'].astype(str).str.startswith('13')|df['fam.status'].astype(str).str.startswith('14')|df['fam.status'].astype(str).str.startswith('15'))
YN = pd.pivot_table(df.loc[filters3],values=['fam.familycode'],index=['SM install status'],aggfunc=pd.Series.nunique,margins=True, observed=True)

#4th pivot table
filtersS = (dfs['unit.status'] == '0-Monitored')
Sources = pd.pivot_table(dfs.loc[filtersS],values=['fam.familycode'],index=['Platforms','src.srctype'],aggfunc='count',margins=True, observed=True)
Sources_Totals = Sources.groupby(by=['Platforms']).sum()


#Extrayendo todos los datos para el reporte final

R1 = TV_Totals.loc['All','fam.familycode']
R2 = TV_Totals.loc['0-Monitored','fam.familycode']
R3 = TV_Totals.loc['1-Temporarily unmonitored','fam.familycode']
R4 = TV_Totals.loc['2-In existance but not be monitored','fam.familycode']
R5 = TV_Totals.loc['3-No longer used','fam.familycode']
R6 = LP.loc['502-PC/Laptop - Sucessfully Mapped','unit.unitid'] + TB.loc['602-Tablet - Successfully Mapped','unit.unitid'] + SP.loc['702-Smartphone - Successfully mapped','unit.unitid']
R7 = LP.loc['503-PC/Laptop - Pending MAC/ODID Mapping','unit.unitid'] + TB.loc['603-Tablet - Pending MAC/ODID Mapping','unit.unitid'] + SP.loc['703-Smartphone - Pending MAC/ODID Mapping','unit.unitid']
R8 = LP.loc['501-PC/Laptop - Waiting Mapping/Registration','unit.unitid'] + TB.loc['601-Tablet-Waiting Mapping/Registration','unit.unitid'] + SP.loc['701-Smartphone-Waiting Mapping/Registration','unit.unitid']
R9 = PT2.loc['8-Digital Device Not Installed','unit.unitid']
R10 = LP.loc['502-PC/Laptop - Sucessfully Mapped','unit.unitid']
R11 = LP.loc['503-PC/Laptop - Pending MAC/ODID Mapping','unit.unitid']
R12 = LP.loc['501-PC/Laptop - Waiting Mapping/Registration','unit.unitid']
R13 = LP2.loc['8-Digital Device Not Installed','unit.unitid']
R14 = TB.loc['602-Tablet - Successfully Mapped','unit.unitid']
R15 = TB.loc['603-Tablet - Pending MAC/ODID Mapping','unit.unitid']
R16 = TB.loc['601-Tablet-Waiting Mapping/Registration','unit.unitid']
R17 = TB2.loc['8-Digital Device Not Installed','unit.unitid']
R18 = SP.loc['702-Smartphone - Successfully mapped','unit.unitid']
R19 = SP.loc['703-Smartphone - Pending MAC/ODID Mapping','unit.unitid']
R20 = SP.loc['701-Smartphone-Waiting Mapping/Registration','unit.unitid']
R21 = SP2.loc['8-Digital Device Not Installed','unit.unitid']
R22 = YN.loc['1 - Yes','fam.familycode']
R23 = Sources_Totals.loc['Digital Terrestrial','fam.familycode']
R24 = Sources_Totals.loc['IPTV/ Internet/ Streaming','fam.familycode']
R25 = Sources_Totals.loc['Satellite','fam.familycode']
R26 = Sources_Totals.loc['Cable','fam.familycode']

report.append(R1)
report.append(R2)
report.append(R3)
report.append(R4)
report.append(R5)
report.append(R6)
report.append(R7)
report.append(R8)
report.append(R9)
report.append(R10)
report.append(R11)
report.append(R12)
report.append(R13)
report.append(R14)
report.append(R15)
report.append(R16)
report.append(R17)
report.append(R18)
report.append(R19)
report.append(R20)
report.append(R21)
report.append(R22)
report.append(R23)
report.append(R24)
report.append(R25)
report.append(R26)

#Creando el dataframe del reporte final
data=[{'Nr. Total TVs':R1,'Nr. Monitored TVs/Metered':R2,'Nr. Temp Unmonitored TVs (Broken)':R3,'Nr. No Monitored - Portable/Only for non-broadcast (DVD etc.)':R4,'Nr. No Monitored - Not used anymore':R5,'Nr Digital Devices Measured':R6,'Nr Digital Devices Temp  Not Measured':R7,'Nr Digital Devices To be Mapped':R8,'Nr Digital Devices Not installed (other reason)':R9,'PC/Laptop - Successfully Mapped':R10,'PC/Laptop - Temp  Not Measured':R11,'PC/Laptop - To be Mapped':R12,'PC/Laptop - Not installed (other reason)':R13,'Tablet - Successfully Mapped':R14,'Tablet - Temp  Not Measured':R15,'Tablet - To be Mapped':R16,'Tablet - Not installed (other reason)':R17,'Smartphone - Successfully Mapped':R18,'Smartphone - Temp  Not Measured':R19,'Smartphone - To be Mapped':R20,'Smartphone - Not installed (other reason)':R21,'Streaming Meter Installed Y/N':R22,'Digital Terrestrial (Y/N)':R23,'IPTV / Internet/Streaming (Y/N)':R24,'Satellite (Y/N)':R25,'Cable (Y/N)':R26}]
df_RF=pd.DataFrame(data, index = ['Total'])

writer = pd.ExcelWriter(f'{path_out}/{date_today}_Output.xlsx')  #path + output_file_name, index = False
df.to_excel(writer,'Historical - Tv units')
dfs.to_excel(writer,'TV_Sources')
df_key.to_excel(writer,'Key')
TV.to_excel(writer,'Pivot 1')
PT.to_excel(writer, 'Pivot 2')
# df_temp_out6.to_excel(writer, 'Output6')
YN.to_excel(writer, 'Pivot 3')
Sources.to_excel(writer, 'Pivot 4')
df_RF.to_excel(writer, 'Devices Registered & Measured')
#df_check10.to_excel(writer, 'Output10')
writer.save()
'''

C:\Users\edsantil2101\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (0,3,4,5,6,7,8,10,11) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
